# Multiparticle MSM/RD simulation

Sample implementation of MSM/RD multiparticle simulation. Full versions in scripts folder. Note it requires to have already computed the main MSM for MSM/RD and have it ready to be pickled by python.

In [ ]:
import numpy as np
import pickle
import msmrd2
import msmrd2
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.markovModels import msmrdMarkovModel as msmrdMSM
from msmrd2.integrators import msmrdMultiParticleIntegrator
import msmrd2.tools.particleTools as particleTools
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis

In [ ]:
# Important paramaters
# Main parameters for particle and integrator
numparticles = 5
partTypes = 0 # All particles are type 0
dt = 0.0001 #0.002 # should be smaller than Gillespie inverse transition rates
bodytype = 'rigidbody'
numBoundStates = 4
maxNumBoundStates = 10
radialBounds = [1.25, 2.25] # must match patchyDimer discretization
minimumUnboundRadius = 1.5
numParticleTypes = 1 # num. of particle types (not states) in unbound state

# Patchy particles parameters
sigma = 1.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]

# Other important parameters
lagtime = 40 #100 #300
boxsize = 6 #2.5 #6 #8 #6
angleDiff = 3*np.pi/5.0
dtMDsimulation = 0.00001
stride = 25
realLagtime = lagtime*dtMDsimulation*stride

# Discretization parameters (need to be consistent with the on used to generate the MSM for MSM/RD
numSphericalSectionsPos = 7 #7 #7
numRadialSectionsQuat = 5 #3 #5
numSphericalSectionsQuat = 7 #6 #7
totalnumSecsQuat = numSphericalSectionsQuat*(numRadialSectionsQuat -1) + 1
numTransitionsStates = numSphericalSectionsPos * totalnumSecsQuat #203

# Parameters to define continuous-time MSM for unbound dynamics: unboundMSM (assumed same for all particles)
MSMtype = 0
ratematrix = np.array([[0]]) # no unbound dynamics
Dlist = np.array([1.0])
Drotlist = np.array([1.0])

# Parameters to define coupling Markov model for bound dynamics: couplingMSM
Dbound = np.ones(numBoundStates)
DboundRot = 0.5*np.ones(numBoundStates)

# Complex diffusion coefficients (D,Drot) (taken from estimateDiffusionCoefficients script)
DlistCompound = np.array([0.6424, 0.2484, 0.06956, 0.0196])
DrotlistCompound = np.array([0.7234, 0.1869, 0.04041, 0.0341])

# Bound states definition, needed to calculate boundstate
boundStates = [1, 2, 3, 4]

In [ ]:
# Define discretization
discretization = msmrd2.discretizations.positionOrientationPartition(radialBounds[1],
                                                                         numSphericalSectionsPos, numRadialSectionsQuat, numSphericalSectionsQuat)

In [ ]:
# Define boundary
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')

In [ ]:
# Load rate dicitionary
pickle_in = open("../../data/pentamer/MSMs/MSM_dimer4trimer_t3.00E+06_s25_lagt" + str(lagtime)
                 +  ".pickle","rb") # Same MSM as trimer
mainMSM = pickle.load(pickle_in)
tmatrix = mainMSM['transition_matrix']
activeSet = mainMSM['active_set']

In [ ]:
# Set unbound MSM
seed = int(-1) # Negative seed, uses random device as seed
unboundMSM = ctmsm(MSMtype, ratematrix, seed)
unboundMSM.setD(Dlist)
unboundMSM.setDrot(Drotlist)

In [ ]:
# Set coupling MSM
seed = int(-1) # Negative seed, uses random device as seed
couplingMSM = msmrdMSM(numBoundStates, maxNumBoundStates,  tmatrix, activeSet, realLagtime, seed)
couplingMSM.setDbound(Dbound, DboundRot)

In [ ]:
# Define integrator, boundary and discretization
seed = -int(1) # Negative seed, uses random device as seed
integrator = msmrdMultiParticleIntegrator(dt, seed, bodytype, numParticleTypes, radialBounds,
                                          unboundMSM, couplingMSM, DlistCompound, DrotlistCompound)
integrator.setBoundary(boxBoundary)
integrator.setDiscretization(discretization)

In [ ]:
partlist = particleTools.randomParticleMSList(numparticles, boxsize,
                                                  minimumUnboundRadius, partTypes, [unboundMSM], seed)

In [ ]:
#Integrate particle list and print only positions 
timesteps = 500000 #100000 #20000000
stride = 25 #250 #1000
datafile  = open('../../data/vmd/multiParticleMSMRD.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%10000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                    outfname = "multiParticleMSMRD", 
                                    tclfname = "../../data/vmd/multiParticleMSMRD_2vmd.tcl")
print("Percentage complete: ", 100, " %")

In [ ]:
# Check if loops were formed.
integrator.findClosedBindingLoops(partlist)